In [ ]:
config_text = u"""
; IMPORTANT: replace these values with those provided by OSIsoft
[Configurations]
Namespace = UC__Davis

[Access]
Resource = https://dat-b.osisoft.com
Tenant = 65292b6c-ec16-414a-b583-ce7ae04046d4
ApiVersion = v1-preview

[Credentials] 
ClientId = 5f77dc80-ad37-4055-bd54-cccf987d9bb5
ClientSecret = *********REPLACE_ME**********
"""

# HackDavis python library tests

## OCS-HackDavis Package Installation 

In [ ]:
!pip install -i https://test.pypi.org/simple/ --extra-index-url https://pypi.org/simple/ ocs-hackdavis==0.34.0

## Import functions from package `ocs_hackdavis`

### They are all described in this notebook with examples

In [ ]:
from ocs_hackdavis import (
    ucdavis_buildings,  # list of campus buildings
    ucdavis_ceeds_of,   # list of CEED element of a building (Electricity, Steam, Chilled Water, etc)
    ucdavis_streams_of, # The list of all OCS data streams for a building and CEED pair  
    ucdavis_building_metadata,  # Metadata for a building: building code, lat/long, usage, etc.  
    ocs_stream_interpolated_data,  # Interpolated data from a stream given a time range + interpolation interval
    ucdavis_outside_temperature,  # Outside temperature at UC Davis for a given a time range + interpolation interval
)

## List of buildings

In [ ]:
buildings = ucdavis_buildings()
len(buildings), buildings[:25]  # display first 25

## Function `ucdavis_ceeds_of` returns list of CEEDs for a given building

### Display them for all buildings

In [ ]:
for num, building in enumerate(ucdavis_buildings(), 1):
    print(f"[{num}] Building: {building} ==> CEEDS: {ucdavis_ceeds_of(building)}")

## Get the list of available streams of a building

#### Default CEED is Electricity

In [ ]:
ucdavis_streams_of("Activities and Recreation Center")

## Get the specific stream of a building

In [ ]:
ucdavis_streams_of("Activities and Recreation Center")["Demand"]

## Import for OCS Sample Library and other necessary modules

In [ ]:
from ocs_sample_library_preview import OCSClient
import configparser
import io
import json
import pandas as pd

## Standard configuration file parsing and OCS client object 

#### Reference: https://github.com/osisoft/OSI-Samples-OCS/blob/master/basic_samples/DataViews/Python3/program.py#L150

## IMPORTANT: REPLACE CLIENT ID + SECRET BELOW BEFORE RUNNING

In [ ]:
config = configparser.ConfigParser(allow_no_value=True)
config.read_file(io.StringIO(config_text))

ocs_client = OCSClient(
    config.get("Access", "ApiVersion"),
    config.get("Access", "Tenant"),
    config.get("Access", "Resource"),
    config.get("Credentials", "ClientId"),
    config.get("Credentials", "ClientSecret"),
)

namespace_id = config.get("Configurations", "Namespace")
print(f"namespace_id: '{namespace_id}'")

## Getting interpolated data from a stream

In [ ]:
# Step 1: get the stream Id 
stream_id = ucdavis_streams_of("Activities and Recreation Center")["Demand"]

# Step 2) request interpolated data
# NOTE 1: difference between endIndex and startIndex should be 31 days or less
# NOTE 2: interpolation interval cannot be less than 2 minutes
result = ocs_stream_interpolated_data(
    ocs_client,
    namespace_id,
    stream_id,
    start="2017-02-01", # UTC 
    end="2017-03-01",
    interval=2,  # 2 minutes
)
len(result), result[:100]  # display first 100 rows 

## Transform result into a Pandas series

In [ ]:
demand1 = pd.read_json(json.dumps(result)).set_index('Timestamp')
demand1

## Get data for next month

In [ ]:
result = ocs_stream_interpolated_data(
    ocs_client,
    namespace_id,
    stream_id,
    start="2017-03-01", # UTC 
    end="2017-04-01",
    interval=2,  # 2 minutes
)

## Transform into a Pandas time series and append to first month data

In [ ]:
demand2 = pd.read_json(json.dumps(result)).set_index('Timestamp')
demand_2m = demand1.append(demand2)
len(demand_2m), demand_2m

## There is a repeated row at the boundary of the two appended series

More specifically, the bottom row of first series and first row of second series. Line below shows it:

In [ ]:
demand_2m.loc[demand_2m.index.duplicated()]

## Remove duplicated row

Difference in lenght (== # of rows) should be 1 

In [ ]:
new_demand = demand_2m.loc[~demand_2m.index.duplicated(keep="first")]
len(demand_2m) - len(new_demand)

## Getting a full year of data

### Reusing what we've learn so far 

In [ ]:
# Create an empty series
demand = pd.Series()
demand

In [ ]:
for start_month in range(1, 12):
    start_date = f"2017-{start_month}-01"
    print(f"> processing {start_date}")
    result = ocs_stream_interpolated_data(
        ocs_client,
        namespace_id,
        stream_id,
        start=start_date,  # UTC
        end=f"2017-{start_month+1}-01",
        interval=2,  # 2 minutes
    )
    demand = demand.append(
        pd.read_json(json.dumps(result)).set_index("Timestamp", drop=True)
    )

# Note there is an additional column "0" with NaN (not a number), it will be remove in the next cell
len(demand), demand

In [ ]:
# remove duplicate rows and keep only column named "Values"
demand = demand.loc[~demand.index.duplicated(keep="first")]["Value"]
len(demand), demand

## Building metadata

In [ ]:
ucdavis_building_metadata(ocs_client, namespace_id, "Activities and Recreation Center")

## Get outside temperature at UC Davis

### Data for this stream starts on 2017-04-01, 19:38

In [ ]:
# display only first 100 rows
ucdavis_outside_temperature(ocs_client, namespace_id, "2017-04-01", "2017-05-01", 2)[:100]

## Test error code 

### Bad argument of the right type

In [ ]:
# should fail
ucdavis_ceeds_of("Bad Building")